<a href="https://www.kaggle.com/code/kapturovalexander/kapturov-s-loan-approval-prediction-s4e10?scriptVersionId=198991221" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# <span style="background-image: url(https://www.kaggle.com/competitions/84894/images/header); color: gold; font-size: 50px; padding: 10px;">Loan Approval Prediction</span>
## <span style="border: solid gold 3px; background-color: #B58050; color: white; padding: 3px">Playground Series - Season 4, Episode 10

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e10/sample_submission.csv
/kaggle/input/playground-series-s4e10/train.csv
/kaggle/input/playground-series-s4e10/test.csv


- #### <span style="color: #B58050">version 1</span> (first glance at the data, without EDA)
- #### <span style="color: #B58050">version 2</span> (I found outliers in "person_emp_length" feature)

![](https://www.kaggle.com/competitions/84894/images/header)

## 🪙 <span style="color: #B58050">Starting from this competition I will return to daily activity on kaggle because I need to become a strong professional in machine learning and data science. I hope this notebook will be the best among my previous ones and my results at the end of the competition will be good!

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# 'id' feature is index
train = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv', index_col=0)
test = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv', index_col=0)

In [4]:
# Display all features in DataFrame
pd.set_option('display.max_columns', None)

In [5]:
train.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
id,,,,,,,,,,,,
0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [6]:
train.tail(3)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
id,,,,,,,,,,,,
58642,23,44000,RENT,7.0,EDUCATION,D,6800,16.00,0.15,N,2,1
58643,22,30000,RENT,2.0,EDUCATION,A,5000,8.90,0.17,N,3,0
58644,31,75000,MORTGAGE,2.0,VENTURE,B,15000,11.11,0.20,N,5,0


- <span style="color: #B58050">**person_age**</span>: Age of the person applying for the loan, measured in years.
- <span style="color: #B58050">**person_income**</span>: Annual income of the applicant in dollars.
- <span style="color: #B58050">**person_home_ownership**</span>: Type of home ownership (e.g., RENT, MORTGAGE, OWN).
- <span style="color: #B58050">**person_emp_length**</span> Length of employment in years (for whole life maybe).
- <span style="color: #B58050">**loan_intent**</span>: Purpose of the loan (e.g., EDUCATION, VENTURE, PERSONAL).
- <span style="color: #B58050">**loan_grade**</span>: Assigned grade to the loan indicating its risk profile (e.g., A is the best).
- <span style="color: #B58050">**loan_amnt**</span>: Amount of money requested by the applicant for the loan.
- <span style="color: #B58050">**loan_int_rate**</span>: Annual interest rate of the loan, shown as a percentage.
- <span style="color: #B58050">**loan_percent_income**</span>: Ratio of the loan amount to the applicant's annual income.
- <span style="color: #B58050">**cb_person_default_on_file**</span>: Indicator of whether the person has a history of defaulting on other credit obligations (Y for Yes, N for No).
- <span style="color: #B58050">**cb_person_cred_hist_length**</span>: Length of the applicant's credit history, measured in years.
- <span style="color: #B58050">**loan_status**</span>: Status of the loan (1 if defaulted, 0 if not).

In [7]:
# For visualization purposes
numeric = ['person_age', 'person_income', 'person_emp_length', 'loat_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length']
categorical = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']

In [8]:
train.shape

(58645, 12)

### <span style="color: #B58050">There are 12 features (one of them is target) and about 59K observations

In [9]:
test.shape

(39098, 11)

![](https://images.ctfassets.net/1aemqu6a6t65/5rPsNLkgpwvZPSmjhE5ChB/9ee0978b3d792515d59fefab5296b832/wall-street-photo-tagger-yancey-iv-nyc-and-company-02-2?w=1200&h=800&q=75)

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58645 entries, 0 to 58644
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  58645 non-null  int64  
 1   person_income               58645 non-null  int64  
 2   person_home_ownership       58645 non-null  object 
 3   person_emp_length           58645 non-null  float64
 4   loan_intent                 58645 non-null  object 
 5   loan_grade                  58645 non-null  object 
 6   loan_amnt                   58645 non-null  int64  
 7   loan_int_rate               58645 non-null  float64
 8   loan_percent_income         58645 non-null  float64
 9   cb_person_default_on_file   58645 non-null  object 
 10  cb_person_cred_hist_length  58645 non-null  int64  
 11  loan_status                 58645 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 5.8+ MB


### <span style="color: #B58050">At first glance, there are no gaps in the data.

In [11]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
person_age,58645.0,27.550857,6.033216,20.00,23.00,26.00,30.00,123.00
person_income,58645.0,64046.172871,37931.106978,4200.00,42000.00,58000.00,75600.00,1900000.00
person_emp_length,58645.0,4.701015,3.959784,0.00,2.00,4.00,7.00,123.00
loan_amnt,58645.0,9217.556518,5563.807384,500.00,5000.00,8000.00,12000.00,35000.00
loan_int_rate,58645.0,10.677874,3.034697,5.42,7.88,10.75,12.99,23.22
loan_percent_income,58645.0,0.159238,0.091692,0.00,0.09,0.14,0.21,0.83
cb_person_cred_hist_length,58645.0,5.813556,4.029196,2.00,3.00,4.00,8.00,30.00
loan_status,58645.0,0.142382,0.349445,0.00,0.00,0.00,0.00,1.00


### <span style="color: #B58050">About 3/4 of borrowers are people in age not above than 30. 75% of sample has income below 76k$.</span>
# <span style="color: #B58050">I also guess that observation where person_age is 123 and person_emp_length is also 123 is anomaly, so don't forget to clear the data!

### My post about this: [🤔 Anomaly or what does means ''person_emp_length'' feature?](https://www.kaggle.com/competitions/playground-series-s4e10/discussion/536990)

# 🪙 <span style="color: #B58050">Outliers detection

In [12]:
# How many years people didn't work
years_without_work = train['person_age'] - train['person_emp_length']
years_without_work.sort_values(ascending=True).head(30)

id
49252   -102.0
41079    -95.0
41912      2.0
26125      2.0
43518      6.0
746        6.0
52038      7.0
24214      8.0
32236      8.0
58602      9.0
18783      9.0
56176      9.0
33012     10.0
699       11.0
12638     11.0
17542     11.0
34852     12.0
48915     12.0
16378     12.0
3072      12.0
48118     12.0
53992     12.0
17861     12.0
15921     13.0
20615     13.0
8222      13.0
18293     13.0
48185     13.0
47532     14.0
52901     14.0
dtype: float64

In [13]:
train.iloc[[49252, 41079]]

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
id,,,,,,,,,,,,
49252,21,192000,MORTGAGE,123.0,VENTURE,B,20000,11.49,0.10,N,2,0
41079,28,60350,MORTGAGE,123.0,MEDICAL,D,25000,15.95,0.35,Y,6,1


In [14]:
outliers_indexes = years_without_work.sort_values(ascending=True).head(28).index
outliers_indexes

Index([49252, 41079, 41912, 26125, 43518,   746, 52038, 24214, 32236, 58602,
       18783, 56176, 33012,   699, 12638, 17542, 34852, 48915, 16378,  3072,
       48118, 53992, 17861, 15921, 20615,  8222, 18293, 48185],
      dtype='int64', name='id')

In [15]:
train.iloc[outliers_indexes]

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
id,,,,,,,,,,,,
49252,21,192000,MORTGAGE,123.0,VENTURE,B,20000,11.49,0.10,N,2,0
41079,28,60350,MORTGAGE,123.0,MEDICAL,D,25000,15.95,0.35,Y,6,1
41912,23,48686,RENT,21.0,EDUCATION,B,12000,11.86,0.25,N,2,0
26125,43,56000,RENT,41.0,HOMEIMPROVEMENT,B,2000,10.99,0.04,N,14,0
43518,25,42000,RENT,19.0,MEDICAL,C,6000,13.49,0.14,N,4,0
746,23,100000,MORTGAGE,17.0,PERSONAL,A,17000,7.49,0.17,N,3,0
52038,38,42000,MORTGAGE,31.0,VENTURE,B,11000,11.49,0.26,N,14,0
24214,43,50000,RENT,35.0,DEBTCONSOLIDATION,A,9000,5.42,0.18,N,16,0
32236,23,25596,RENT,15.0,EDUCATION,A,5000,7.90,0.20,N,4,0


### <span style="color: #B58050">I don't know at what age people in the US or other countries can start working, but I'm guessing that the 28 observations above are outliers, since their years_out_of_work are less than 14 years.

In [16]:
train.describe(include=['object']).T

,count,unique,top,freq
person_home_ownership,58645,4,RENT,30594
loan_intent,58645,6,EDUCATION,12271
loan_grade,58645,7,A,20984
cb_person_default_on_file,58645,2,N,49943
